Based off of work in vocab.ipynb, this program extracts tobacco and drug related bigrams from each text in a given time period. 

In [1]:
from collections import Counter
import os
import pandas as pd
from gensim.models import Phrases, phrases

def getTexts(folder,searchList):
    fileToText = {}
    underscores = {}
    for root, dirs, files in os.walk(folder):
        for file in files:
                if '.txt' not in file: continue
                path = os.path.join(folder,file)
                f = open(path,'r')
                text = f.readlines()[0]
                if '_' in file: 
                        name = file.split('_')[0]
                        if name not in searchList: continue
                        if name not in underscores.keys(): 
                                underscores[name] = text
                        else: underscores[name] = underscores[name] + ' ' + text
                else: 
                        name = file.split('.')[0]
                        if name not in searchList: continue
                        fileToText[name] = text
                f.close()
        for name,text in underscores.items():
            fileToText[name] = text
        return fileToText

def collectgrams(keylist, searchword, collectlist):
        for key in keylist:
                if searchword in key.split('_'):
                        collectlist.append(key) 

def getgrams(txt):
    infile = open(txt,'r')
    lines = infile.readlines()
    infile.close()
    allgrams = []
    for line in lines: 
        line = line.split(':')
        ngrams = line[1].strip()
        if '_' in ngrams: 
                ngrams = ngrams.strip().split(' ')
                for n in ngrams:
                        allgrams.append(n)
        else: continue
    return allgrams 

In [2]:
#change for each time period: index is period - 1
period = open('/srv/data/timeranges.txt', 'r').readlines()[3]
period = period.strip().strip('[').strip(']').replace("'", '').split(', ')
empty = []

bigramdata = getTexts('/srv/data/relevantEPBodyNOSTOP', period)
bigramtexts = list(bigramdata.values())
bigramnames = list(bigramdata.keys())
print(len(bigramnames))

1351


In [40]:
bigrammodel = Phrases(bigramtexts, min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('bigram model trained')
trigrammodel = Phrases(bigrammodel[bigramtexts], min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('trigram model trained')

bigram model trained
trigram model trained


In [41]:
#specify tobacco/drugs searchword list:
# searchwords = ['tobacco']
searchwords = ['opiate', 'opium', 'poppy', 'poppey', 'laudanum']

count = 0
outfile = open('/srv/data/joy/what.txt', 'w')
for text in bigramtexts:
    #for outputting to txt file, specify here
    name = bigramnames[bigramtexts.index(text)]
    testtext = text.strip().split(' ')
    grams = []

    bgcount = Counter(b for b in bigrammodel[text] if len(b.split("_")) > 1)
    tgcount = Counter(t for t in trigrammodel[text] if len(t.split("_")) > 2)
   
    for searchword in searchwords: 
        collectgrams(dict(bgcount).keys(), searchword, grams) 
        collectgrams(dict(tgcount).keys(), searchword, grams)
    
    gramstring = ' '.join(grams)
    outfile.write(name+': '+gramstring+"\n")
    count += 1
    if count%50 == 0:
        print(count, "files processed")
outfile.close()
print('processing complete')

50 files processed
100 files processed
150 files processed
200 files processed
250 files processed
300 files processed
350 files processed
400 files processed
450 files processed
500 files processed


In [ ]:
p4tobacco = getgrams('/srv/data/periodFeatures/period4features/newtobaccograms.txt')
p1tobacco = getgrams('/srv/data/periodFeatures/period1features/period1tobaccongrams.txt')
p2tobacco = getgrams('/srv/data/periodFeatures/period2features/period2tobaccongrams.txt')
p3tobacco = getgrams('/srv/data/periodFeatures/period3features/period3tobaccongrams.txt')
p5tobacco = getgrams('/srv/data/periodFeatures/period5features/period5tobaccongrams.txt')

alltobacco = getgrams('/srv/data/joy/newtobacco.txt')

In [ ]:
megalist = p4tobacco + p1tobacco + p2tobacco + p3tobacco + p5tobacco

print('uniquely in whole set:', set(alltobacco) - set(megalist))
print('uniquely in thirds set:', set(megalist) - set(alltobacco))


# check = 'attendance'
# for word in p1tobacco:
#     if check in word:
#         print ('word found in period 1')
# for word in p2tobacco:
#     if check in word:
#         print ('word found in period 2')
# for word in p3tobacco:
#     if check in word:
#         print ('word found in period 3')
# for word in p4tobacco:
#     if check in word:
#         print ('word found in period 4')
# for word in p5tobacco:
#     if check in word:
#         print ('word found in period 5')


In [ ]:
plant = ['call', 'clean', 'common', 'description', 'dry', 'green', 'growth', 'kind', 'leaf', 'manner', 'name', 'part', 'petum', 'piece', 'plant', 'property', 'quality', 'root', 'seed', 'small', 'sophisticate', 'sort', 'strong']
medical = ['taste', 'loosen', 'warm', 'hot', 'set', 'almanac', 'cold', 'alymbecke', 'apply', 'bruise', 'suffumigation', 'cane', 'cure', 'draw', 'dress',  'drop', 'head', 'heat', 'help', 'humidity', 'humour', 'infusion', 'oil', 'perfume', 'phlegm', 'powder', 'prepare', 'pudding', 'purge', 'stillify', 'stir',  'vapour', 'water']
recreation = ['face', 'arm', 'ash', 'attendance', 'box', 'break', 'breath', 'cast', 'child', 'consume', 'custom', 'cut', 'drawer',  'forget', 'fume', 'glass', 'grant', 'hard', 'have', 'invention', 'keep', 'know', 'leave', 'light', 'make', 'mark', 'paper', 'pipe', 'present', 'rake', 'savour', 'send', 'sit', 'smell', 'smoke', 'stink', 'strew', 'take', 'taker', 'touch', 'unseal', 'use', 'whiff']
region = ['brazilian', 'dutch', 'england', 'english', 'foreign', 'indian', 'juice',  'lemnos', 'society', 'sovereign', 'sovereignty', 'spanish', 'trinidada', 'virginia']
trade = ['alehouse', 'bag', 'barter', 'bring', 'buy', 'case', 'chest', 'commodity', 'employment', 'furnish', 'gain', 'gift', 'give', 'half', 'house', 'import', 'importation', 'licence', 'line', 'monopoly', 'ounce', 'piece', 'pocket', 'pound', 'quantity', 'retail', 'sell', 'seller', 'ship', 'shop', 'spend', 'store', 'tavern', 'trade', 'trial', 'weight']
commodity = ['ale', 'beer', 'butter', 'chandler', 'copper', 'corn', 'cucumber', 'cup', 'drink', 'flax', 'hemlock', 'herb', 'liquorice', 'nightshade', 'other', 'rice', 'sack', 'starch', 'suet', 'sugar', 'sugarcane', 'weed', 'wine']
malegen = ['bartholomew', 'boy', 'charles', 'chief', 'doctor', 'farmer', 'gentleman', 'his', 'main', 'maker', 'man', 'our', 'own', 'their', 'they', 'thou', 'thy', 'you', 'your']
female = [ 'lady', 'she', 'woman'] 
moral = ['abuse', 'against', 'bait', 'banish', 'best', 'commend', 'concern', 'counterblaste', 'delectable', 'divine', 'drone', 'entitle', 'exceed', 'excellent', 'famous', 'fast', 'few', 'force', 'good', 'great', 'hurt', 'hurtful', 'ignorant', 'immoderate', 'ingrate', 'lack', 'like', 'little', 'love', 'malady', 'many', 'mighty', 'mistake', 'more', 'much', 'must', 'noble', 'ordinary', 'ought', 'pity', 'poison', 'poor', 'power', 'praise', 'precious', 'profane', 'prove', 'reason', 'require', 'resist', 'respectless', 'right', 'save', 'scarce', 'sin', 'sleight', 'some', 'succour', 'such', 'trick', 'unwholesome', 'virtue', 'vow', 'want', 'well', 'wholesome', 'worse', 'yes']
misc = ['far', 'one', 'two', 'about', 'after', 'afterward', 'again', 'all', 'almost', 'also', 'although', 'among', 'any', 'because', 'become', 'before', 'bind', 'both', 'bring', 'but', 'can', 'cause', 'come', 'dru', 'each', 'either', 'engender', 'especial', 'etc', 'even', 'ever', 'fac', 'fill', 'find', 'first', 'fit', 'former', 'forsooth', 'forthwith', 'from', 'here', 'heretofore', 'how', 'if', 'into', 'itself', 'let', 'likewise', 'may', 'mean', 'neither', 'nor', 'not', 'nothing', 'now', 'often', 'once', 'only', 'out', 'ready', 'regard', 'return', 'same', 'say', 'see', 'seem', 'ser', 'shall', 'still', 'sure', 'term', 'than', 'that', 'then', 'there', 'thereby', 'therefore', 'therewith', 'these', 'thing', 'think', 'this', 'those', 'though', 'through', 'till', 'tob', 'too', 'unless', 'upon', 'utter', 'very', 'what', 'whatsoever', 'when', 'where', 'whereas', 'wherein', 'wherewith', 'whether', 'which', 'while', 'who', 'why', 'will', 'within', 'yet']

In [ ]:
print('plant:', sorted(set(plant)))
print('medical:', sorted(set(medical)))
print('use:', sorted(set(recreation)))
print('region:', sorted(set(region)))
print('trade:', sorted(set(trade)))
print('commodity:', sorted(set(commodity)))
print('male/general people:', sorted(set(malegen)))
print('female:', sorted(set(female)))
print('moral:', sorted(set(moral)))
print('misc:', sorted(set(misc)))


In [ ]:
plantgrams = []
femalegrams = []
recreationgrams = []
medicalgrams = []
regiongrams = []
tradegrams = []
commoditygrams = []
moralgrams = []
malegengrams = []
miscgrams = []
leftover = []

for ngram in set(megalist):
    for word in ngram.split('_'):
        if word in malegen:
            malegengrams.append(ngram)
            break
        if word in female:
            femalegrams.append(ngram)
        elif word in plant:
            plantgrams.append(ngram)
            break
        elif word in medical:
            medicalgrams.append(ngram)
        elif word in recreation:
            recreationgrams.append(ngram)
            break
        elif word in region:
            regiongrams.append(ngram)
            break
        elif word in trade:
            tradegrams.append(ngram)
            break
        elif word in commodity:
            commoditygrams.append(ngram)
            break 
        elif word in moral:
            moralgrams.append(ngram)
            break
        elif word in misc:
            miscgrams.append(ngram)
            break
        else: 
            continue

# print('plant:', plantgrams)
# print('use:', usegrams)
# print('region:', regiongrams)
# print('trade:', tradegrams)
# print('commodity:', commoditygrams)
# print('moral:', moralgrams)
# print('people:', peoplegrams)
# print('function:', miscgrams)
# print('leftover:', leftover)

gramdict = {'plant': plantgrams, 'use': recreationgrams, 'medical': medicalgrams, 'region': regiongrams, 'trade': tradegrams, 'commodity': commoditygrams, 'moral': moralgrams, 'female': femalegrams, 'male/general people': malegengrams, 'misc': miscgrams}
# outfile = open('/srv/data/joy/p4groupedtobacco.txt', 'a+')

# for key in gramdict:
#     outfile.write(key+': '+', '.join(gramdict[key])+"\n")
# outfile.close()

print(gramdict)

# categorised = []
# for key in gramdict:
#     for word in gramdict[key]:
#         categorised.append(word)

# leftover = set(megalist) - set(categorised)
# print(leftover)


In [ ]:
dictoutfile = open('/srv/data/joy/ngramdict.txt', 'a+')
dictoutfile.write('Plant: '+', '.join(set(plantgrams))+"\n")
dictoutfile.write('Male/general: '+', '.join(set(malegengrams))+"\n")
dictoutfile.write('Female: '+', '.join(set(femalegrams))+"\n")
dictoutfile.write('Usage: '+', '.join(set(recreationgrams))+"\n")
dictoutfile.write('Medical: '+', '.join(set(medicalgrams))+"\n")
dictoutfile.write('Regions: '+', '.join(set(regiongrams))+"\n")
dictoutfile.write('Trade: '+', '.join(set(tradegrams))+"\n")
dictoutfile.write('Commodity: '+', '.join(set(commoditygrams))+"\n")
dictoutfile.write('Ethics: '+', '.join(set(moralgrams))+"\n")
dictoutfile.write('Miscellaneous: '+', '.join(set(miscgrams))+"\n")

dictoutfile.close()